# 1. Redis Cache for LangChain
https://python.langchain.com/docs/integrations/caches/redis_llm_caching/

In [9]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


In [10]:
import time

from langchain_redis import RedisCache
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache

In [11]:
# Initialize RedisCache
redis_cache = RedisCache(redis_url=REDIS_URL)

# Set the cache for LangChain to use
set_llm_cache(redis_cache)

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini" ,temperature=0)

# Function to measure execution time
def timed_completion(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time


# First call (not cached)
prompt = "Explain the concept of caching in three sentences."
result1, time1 = timed_completion(prompt)
print(f"First call (not cached):\nResult: {result1}\nTime: {time1:.2f} seconds\n")

# Second call (should be cached)
result2, time2 = timed_completion(prompt)
print(f"Second call (cached):\nResult: {result2}\nTime: {time2:.2f} seconds\n")

print(f"Speed improvement: {time1 / time2:.2f}x faster")

# Clear the cache
redis_cache.clear()
print("Cache cleared")

First call (not cached):
Result: content='Caching is a technique used to store frequently accessed data in a temporary storage area, known as a cache, to improve retrieval speed and reduce latency. By keeping copies of data closer to the user or application, caching minimizes the need to repeatedly fetch the same information from slower storage or remote servers. This results in enhanced performance and efficiency, particularly in web applications and databases, where quick access to data is crucial.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 16, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-84f8884b-c854-4204-a

In [12]:
import redis

r = redis.Redis()
r.execute_command('JSON.SET', 'test_key', '.', '{"key": "value"}')
print(r.execute_command('JSON.GET', 'test_key'))


b'{"key":"value"}'
